## Example interference

In [ ]:
#!pip install hackernews-client -q

In [53]:
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
num_labels = 14



model_checkpoint = f"./finetuned/checkpoint-80344/"
model_test = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                                num_labels=num_labels)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


In [54]:
def softmax(_outputs):
    maxes = np.max(_outputs, axis=-1, keepdims=True)
    shifted_exp = np.exp(_outputs - maxes)
    return shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)

def test_interference(trained_model,text):
  text_pt = tokenizer([text],
                    padding="max_length",max_length=201,
                    truncation=True,return_tensors="pt")
  #return np.argmax(model_test(**text_pt)[0][0].detach().numpy())
  return softmax(trained_model(**text_pt)[0][0].detach().numpy())

In [55]:
reduced_cats = {
  "CULTURE & ARTS":["ARTS","ARTS & CULTURE","CULTURE & ARTS"],
 "BUSINESS": ["BUSINESS","MONEY"],
 "EDUCATION": ["EDUCATION","COLLEGE"],
 "COMEDY & ENTERTAINMENT" : ["COMEDY","ENTERTAINMENT","MEDIA"],
 "HEALTH & LIVING": ["WELLNESS","HEALTHY LIVING",
                     "STYLE & BEAUTY","HOME & LIVING",
                     "PARENTS","STYLE","FOOD & DRINK","TASTE","PARENTING","DIVORCE","WEDDINGS"],
 "RELIGION" : ["RELIGION"],
 "POLITICS" : ["POLITICS","BLACK VOICES","LATINO VOICES","QUEER VOICES","WOMEN"],
 "SPORTS" : ["SPORTS"],
 "TRAVEL" : ["TRAVEL"],
 "NEWS" :["GOOD NEWS","THE WORLDPOST","WORLDPOST","WORLD NEWS","WEIRD NEWS","CRIME"],
 "ENVIRONMENT" : ["GREEN","ENVIRONMENT"],
 "SCIENCE": ["SCIENCE"],
 "TECH": ["TECH"],
 "OTHER" : ["IMPACT","FIFTY"]
 }

label_map = {}
for i,cats in enumerate(reduced_cats.values()):
  for cat in cats:
    label_map[cat] = i

num_labels = len(reduced_cats.keys())

lookup_label = {i:v for i,v in enumerate(reduced_cats.values())}

In [56]:
give_cat = lambda x: lookup_label[np.argmax(test_interference(model_test,x))]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/Users/franz/miniforge3/bin/python3.9 -m pip install --upgrade pip' command.


In [57]:
from hackernews import hn
import requests
import re
from lxml import html
news_client = hn.NewsClient()

def get_comm(x):
  try:
    c_id = x.kids[0]
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{c_id}.json").json()['text']
  except:
    return ""


h = news_client.get_best_story(fetchMax=200)



In [38]:
def get_title_and_comment(item)
    cleaned_top_comment =  re.sub('<[^<]+?>',get_comm(x)," ")
    return x.title + "\n" + cleaned_top_comment


headline_and_comment  =  [get_title_and_comment(x) for x in h]


In [59]:

df = pd.DataFrame({
"story": headline_and_comment
})
df["label"] = df["story"].apply(give_cat)

In [61]:
df.sample(10)

,story,label
97,An old photo of a large BBS\n,"[WELLNESS, HEALTHY LIVING, STYLE & BEAUTY, HOM..."
48,Gov.uk Platform as a Service\n,"[POLITICS, BLACK VOICES, LATINO VOICES, QUEER ..."
115,Orcas found to kill blue whales\n,"[GREEN, ENVIRONMENT]"
15,Prime Video Uses WebAssembly\n,[TECH]
155,"Walking America: Washington, DC (Anacostia and...",[TRAVEL]
40,Show HN: Hibiki HTML – New frontend framework ...,[TECH]
191,Arizona Introduces a bill to make Bitcoin Lega...,"[BUSINESS, MONEY]"
79,Decryption through LUKS2 reencryption crash re...,"[GOOD NEWS, THE WORLDPOST, WORLDPOST, WORLD NE..."
19,We have jetpacks and we do not care\n,"[GOOD NEWS, THE WORLDPOST, WORLDPOST, WORLD NE..."
165,Choosing tools for Game Boy development\n,"[BUSINESS, MONEY]"
